In [1]:
import numpy as np
import pandas as pd
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('BigBasket Products.csv')
df.head()

,Unnamed: 0,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description,quantity
0,0,1,garlic oil - vegetarian capsule 500 mg,beauty & hygiene,hair care,sri sri ayurveda,220.0,220.0,hair oil & serum,4.1,This Product contains Garlic Oil that is known...,500 mg
1,1,2,water bottle - orange,"kitchen, garden & pets",storage & accessories,mastercook,180.0,180.0,water & fridge bottles,2.3,"Each product is microwave safe (without lid), ...",Unknown
2,2,3,"brass angle deep - plain, no.2",cleaning & household,pooja needs,trm,119.0,250.0,lamp & lamp oil,3.4,"A perfect gift for all occasions, be it your m...",Unknown
3,3,4,cereal flip lid container/storage jar - assort...,cleaning & household,bins & bathroom ware,nakoda,149.0,176.0,"laundry, storage baskets",3.7,Multipurpose container with an attractive desi...,Unknown
4,4,5,creme soft soap - for hands & body,beauty & hygiene,bath & hand wash,nivea,162.0,162.0,bathing bars & soaps,4.4,Nivea Creme Soft Soap gives your skin the best...,Unknown


In [3]:
len(df)

27553

In [4]:
df.describe()

,Unnamed: 0,index,sale_price,market_price,rating
count,27553.000000,27553.000000,27553.000000,27553.000000,27553.000000
mean,13777.124342,13778.124342,322.529145,382.073872,3.992418
std,7954.838872,7954.838872,486.277432,581.747762,0.616860
min,0.000000,1.000000,2.450000,3.000000,1.000000
25%,6888.000000,6889.000000,95.000000,100.000000,4.000000
50%,13777.000000,13778.000000,190.000000,220.000000,4.100000
75%,20666.000000,20667.000000,359.000000,425.000000,4.200000
max,27554.000000,27555.000000,12500.000000,12500.000000,5.000000


In [5]:
df.isna().sum()

Unnamed: 0      0
index           0
product         0
category        0
sub_category    0
brand           0
sale_price      0
market_price    0
type            0
rating          0
description     0
quantity        0
dtype: int64

In [6]:
df.isna().sum()/ len(df)*100

Unnamed: 0      0.0
index           0.0
product         0.0
category        0.0
sub_category    0.0
brand           0.0
sale_price      0.0
market_price    0.0
type            0.0
rating          0.0
description     0.0
quantity        0.0
dtype: float64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27553 entries, 0 to 27552
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    27553 non-null  int64  
 1   index         27553 non-null  int64  
 2   product       27553 non-null  object 
 3   category      27553 non-null  object 
 4   sub_category  27553 non-null  object 
 5   brand         27553 non-null  object 
 6   sale_price    27553 non-null  float64
 7   market_price  27553 non-null  float64
 8   type          27553 non-null  object 
 9   rating        27553 non-null  float64
 10  description   27553 non-null  object 
 11  quantity      27553 non-null  object 
dtypes: float64(3), int64(2), object(7)
memory usage: 2.5+ MB


In [8]:
df.columns

Index(['Unnamed: 0', 'index', 'product', 'category', 'sub_category', 'brand',
       'sale_price', 'market_price', 'type', 'rating', 'description',
       'quantity'],
      dtype='object')

In [9]:
df.columns = df.columns.str.strip()


In [10]:
df['rating'] = df['rating'].fillna(df['rating'].median())

In [11]:
df.isnull().sum()

Unnamed: 0      0
index           0
product         0
category        0
sub_category    0
brand           0
sale_price      0
market_price    0
type            0
rating          0
description     0
quantity        0
dtype: int64

In [12]:
df['description'] = df['description'].fillna("No description available")

In [13]:
df.isnull().sum()

Unnamed: 0      0
index           0
product         0
category        0
sub_category    0
brand           0
sale_price      0
market_price    0
type            0
rating          0
description     0
quantity        0
dtype: int64

In [14]:
df = df.dropna(subset=['brand'])

In [15]:
df.isnull().sum()

Unnamed: 0      0
index           0
product         0
category        0
sub_category    0
brand           0
sale_price      0
market_price    0
type            0
rating          0
description     0
quantity        0
dtype: int64

In [16]:
df = df.dropna(subset=['product'])

In [17]:
df.isnull().sum()

Unnamed: 0      0
index           0
product         0
category        0
sub_category    0
brand           0
sale_price      0
market_price    0
type            0
rating          0
description     0
quantity        0
dtype: int64

In [18]:
# Check duplicates values
df.duplicated().sum()

np.int64(0)

In [19]:
# Check inconsistent values.
df['brand'] = df['brand'].str.strip().str.lower()

In [20]:
df['product'] = df['product'].str.strip().str.lower()


In [21]:
df['category'] = df['category'].str.strip().str.lower()

In [22]:
df['sub_category'] = df['sub_category'].str.strip().str.lower()


In [23]:
df['type'] = df['type'].str.strip().str.lower()

In [24]:
df.head()

,Unnamed: 0,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description,quantity
0,0,1,garlic oil - vegetarian capsule 500 mg,beauty & hygiene,hair care,sri sri ayurveda,220.0,220.0,hair oil & serum,4.1,This Product contains Garlic Oil that is known...,500 mg
1,1,2,water bottle - orange,"kitchen, garden & pets",storage & accessories,mastercook,180.0,180.0,water & fridge bottles,2.3,"Each product is microwave safe (without lid), ...",Unknown
2,2,3,"brass angle deep - plain, no.2",cleaning & household,pooja needs,trm,119.0,250.0,lamp & lamp oil,3.4,"A perfect gift for all occasions, be it your m...",Unknown
3,3,4,cereal flip lid container/storage jar - assort...,cleaning & household,bins & bathroom ware,nakoda,149.0,176.0,"laundry, storage baskets",3.7,Multipurpose container with an attractive desi...,Unknown
4,4,5,creme soft soap - for hands & body,beauty & hygiene,bath & hand wash,nivea,162.0,162.0,bathing bars & soaps,4.4,Nivea Creme Soft Soap gives your skin the best...,Unknown


In [25]:
df['product'].nunique()

23510

In [26]:
df[df.duplicated(subset='product', keep=False)].sort_values(by='product')

,Unnamed: 0,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description,quantity
21398,21400,21401,1-2-3 noodles - chicken flavour,snacks & branded foods,"noodle, pasta, vermicelli",wai wai,12.0,12.0,instant noodles,4.2,"Wai Wai, the instant way to mouthwatering nood...",Unknown
20239,20241,20242,1-2-3 noodles - chicken flavour,snacks & branded foods,snacks & namkeen,wai wai,12.0,12.0,instant noodles,4.2,"Wai Wai, the instant way to mouthwatering nood...",Unknown
25401,25403,25404,1-2-3 noodles - pure vegetarian,snacks & branded foods,"noodle, pasta, vermicelli",wai wai,28.0,35.0,instant noodles,4.0,Wai Wai-the instant way to mouthwatering noodl...,Unknown
17370,17372,17373,1-2-3 noodles - pure vegetarian,snacks & branded foods,snacks & namkeen,wai wai,28.0,35.0,instant noodles,4.0,Wai Wai-the instant way to mouthwatering noodl...,Unknown
9485,9485,9486,1-2-3 noodles - veg masala flavour,snacks & branded foods,"noodle, pasta, vermicelli",wai wai,12.0,12.0,instant noodles,4.2,"Wai Wai, the instant way to mouthwatering nood...",Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
10301,10302,10303,zigzag anti-bacterial toothbrush - soft,beauty & hygiene,oral care,colgate,70.0,70.0,toothbrush,4.2,"Colgate Zig Zag, one of India’s leading toothb...",Unknown
9636,9636,9637,zucchini - green,fruits & vegetables,exotic fruits & veggies,fresho,50.0,62.5,exotic vegetables,4.1,"Zucchini is a long, slender, cucumber like gre...",Unknown
333,333,334,zucchini - green,fruits & vegetables,fresh vegetables,fresho,50.0,62.5,exotic vegetables,4.1,"Zucchini is a long, slender, cucumber like gre...",Unknown
6391,6391,6392,zucchini - yellow,fruits & vegetables,fresh vegetables,fresho,50.0,62.5,exotic vegetables,4.1,Yellow zucchini is a long and slightly fat veg...,Unknown


In [27]:
df.groupby(by = 'sub_category').count()

,Unnamed: 0,index,product,category,brand,sale_price,market_price,type,rating,description,quantity
sub_category,,,,,,,,,,,
all purpose cleaners,475,475,475,475,475,475,475,475,475,475,475
appliances & electricals,138,138,138,138,138,138,138,138,138,138,138
"atta, flours & sooji",152,152,152,152,152,152,152,152,152,152,152
baby accessories,37,37,37,37,37,37,37,37,37,37,37
baby bath & hygiene,206,206,206,206,206,206,206,206,206,206,206
...,...,...,...,...,...,...,...,...,...,...,...
steel utensils,353,353,353,353,353,353,353,353,353,353,353
storage & accessories,1015,1015,1015,1015,1015,1015,1015,1015,1015,1015,1015
tea,334,334,334,334,334,334,334,334,334,334,334


In [28]:
df['product'].duplicated().sum()

np.int64(4043)

In [29]:
# df = df.drop_duplicates(subset=['product', 'category', 'sub_category'], keep='first')

In [30]:
df['category'].duplicated().sum()

np.int64(27542)

In [31]:
df['type'].duplicated().sum()

np.int64(27127)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27553 entries, 0 to 27552
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    27553 non-null  int64  
 1   index         27553 non-null  int64  
 2   product       27553 non-null  object 
 3   category      27553 non-null  object 
 4   sub_category  27553 non-null  object 
 5   brand         27553 non-null  object 
 6   sale_price    27553 non-null  float64
 7   market_price  27553 non-null  float64
 8   type          27553 non-null  object 
 9   rating        27553 non-null  float64
 10  description   27553 non-null  object 
 11  quantity      27553 non-null  object 
dtypes: float64(3), int64(2), object(7)
memory usage: 2.5+ MB


In [33]:
len(df)

27553

In [34]:
df.groupby('product').agg({
    'brand': 'nunique',
    'sub_category': 'nunique',
    'sale_price': ['min', 'max', 'mean'],
    'market_price': ['min', 'max', 'mean']
}).reset_index()


product   brand sub_category  \
                                                         nunique      nunique   
0                                   & moms - citrus soap       1            1   
1                                   & moms - sandal soap       1            1   
2                          & moms bathing soap - jasmine       1            1   
3                            & moms bathing soap - tulsi       1            1   
4                                 0.0 non alcoholic beer       1            1   
...                                                  ...     ...          ...   
23505  zippy extra soft toothbrush for kids - red, ag...       1            1   
23506  zookers animal shaped biscuits - cherry bits, ...       1            1   
23507                zoom ultimate dark triangle pencils       1            1   
23508                                   zucchini - green       1            2   
23509                                  zucchini - yellow       1            2   

      sale_price                 market_price                
             min     max    mean          min    max   mean  
0         195.00  195.00  195.00        195.0  195.0  195.0  
1         108.00  108.00  108.00        120.0  120.0  120.0  
2         108.00  108.00  108.00        120.0  120.0  120.0  
3         108.00  108.00  108.00        120.0  120.0  120.0  
4          56.25   56.25   56.25         75.0   75.0   75.0  
...          ...     ...     ...          ...    ...    ...  
23505      40.00   40.00   40.00         40.0   40.0   40.0  
23506     160.00  160.00  160.00        160.0  160.0  160.0  
23507     147.00  147.00  147.00        150.0  150.0  150.0  
23508      50.00   50.00   50.00         62.5   62.5   62.5  
23509      50.00   50.00   50.00         62.5   62.5   62.5  

[23510 rows x 9 columns]

In [35]:
df_grouped = df.groupby('product').agg({
    'brand': lambda x: ', '.join(sorted(set(x))),
    'sub_category': lambda x: ', '.join(sorted(set(x))),
    'sale_price': 'mean',
    'market_price': 'mean',
    'rating': 'mean'
}).reset_index()
df_grouped

,product,brand,sub_category,sale_price,market_price,rating
0,& moms - citrus soap,dettol,bath & hand wash,195.00,195.0,4.2
1,& moms - sandal soap,dettol,bath & hand wash,108.00,120.0,4.2
2,& moms bathing soap - jasmine,dettol,bath & hand wash,108.00,120.0,4.6
3,& moms bathing soap - tulsi,dettol,bath & hand wash,108.00,120.0,4.4
4,0.0 non alcoholic beer,heineken,energy & soft drinks,56.25,75.0,4.2
...,...,...,...,...,...,...
23505,"zippy extra soft toothbrush for kids - red, ag...",dentoshine,oral care,40.00,40.0,3.6
23506,"zookers animal shaped biscuits - cherry bits, ...",timios,baby food & formula,160.00,160.0,3.9
23507,zoom ultimate dark triangle pencils,doms,stationery,147.00,150.0,3.9
23508,zucchini - green,fresho,"exotic fruits & veggies, fresh vegetables",50.00,62.5,4.1


In [36]:
# Function to extract quantity/size from product name
def extract_quantity(text):
    pattern = r'(\d+\.?\d*)\s*(ml|l|g|kg|gm|litre|liter|pcs|pieces|tablet|capsule|mg)'
    matches = re.findall(pattern, str(text).lower())
    return ' '.join([' '.join(match) for match in matches]) if matches else None

# Apply the function to create a new column
df['quantity'] = df['product'].apply(extract_quantity)

# Show a sample of the updated DataFrame with extracted quantities
df[['product', 'quantity']].head(10)

,product,quantity
0,garlic oil - vegetarian capsule 500 mg,500 mg
1,water bottle - orange,None
2,"brass angle deep - plain, no.2",None
3,cereal flip lid container/storage jar - assort...,None
4,creme soft soap - for hands & body,None
5,germ - removal multipurpose wipes,None
6,multani mati,None
7,hand sanitizer - 70% alcohol base,None
8,biotin & collagen volumizing hair shampoo + bi...,None
9,"scrub pad - anti- bacterial, regular",None


In [37]:
df.head()

,Unnamed: 0,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description,quantity
0,0,1,garlic oil - vegetarian capsule 500 mg,beauty & hygiene,hair care,sri sri ayurveda,220.0,220.0,hair oil & serum,4.1,This Product contains Garlic Oil that is known...,500 mg
1,1,2,water bottle - orange,"kitchen, garden & pets",storage & accessories,mastercook,180.0,180.0,water & fridge bottles,2.3,"Each product is microwave safe (without lid), ...",None
2,2,3,"brass angle deep - plain, no.2",cleaning & household,pooja needs,trm,119.0,250.0,lamp & lamp oil,3.4,"A perfect gift for all occasions, be it your m...",None
3,3,4,cereal flip lid container/storage jar - assort...,cleaning & household,bins & bathroom ware,nakoda,149.0,176.0,"laundry, storage baskets",3.7,Multipurpose container with an attractive desi...,None
4,4,5,creme soft soap - for hands & body,beauty & hygiene,bath & hand wash,nivea,162.0,162.0,bathing bars & soaps,4.4,Nivea Creme Soft Soap gives your skin the best...,None


In [38]:
df.groupby(['product', 'quantity']).size().sort_values(ascending=False).head(10)

product                                                                           quantity                
durum wheat pasta 500g + green olives 450g + olive oil 500ml + pasta sauce 300g   500 g 450 g 500 ml 300 g    4
cow ghee/tuppa - a2 gir bilona                                                    2 g                         4
san remo pasta 500g + disano olive oil pure 1l +american garden pasta sauce 397g  500 g 1 l 397 g             3
san remo pasta 500g + disano olive oil 500ml + american garden pasta sauce 88ml   500 g 500 ml 88 ml          3
ezee 2-in-1 liquid detergent + fabric conditioner                                 1 l                         3
idli - sooji 2kg + sugar 2kg + peanuts/ mungaphali/shengdana - raw 500gm          2 kg 2 kg 500 g             3
bb royal sona masoori raw rice 10kg + toor dal 1kg + sugar 1kg                    10 kg 1 kg 1 kg             3
happy chef pasta- penne rigate 500g +sauce - arrabiata 270g +borges olive oil 1l  500 g 270 g 1 l            

In [39]:
df['quantity'] = df['quantity'].fillna("Unknown")

In [40]:
df.head()

,Unnamed: 0,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description,quantity
0,0,1,garlic oil - vegetarian capsule 500 mg,beauty & hygiene,hair care,sri sri ayurveda,220.0,220.0,hair oil & serum,4.1,This Product contains Garlic Oil that is known...,500 mg
1,1,2,water bottle - orange,"kitchen, garden & pets",storage & accessories,mastercook,180.0,180.0,water & fridge bottles,2.3,"Each product is microwave safe (without lid), ...",Unknown
2,2,3,"brass angle deep - plain, no.2",cleaning & household,pooja needs,trm,119.0,250.0,lamp & lamp oil,3.4,"A perfect gift for all occasions, be it your m...",Unknown
3,3,4,cereal flip lid container/storage jar - assort...,cleaning & household,bins & bathroom ware,nakoda,149.0,176.0,"laundry, storage baskets",3.7,Multipurpose container with an attractive desi...,Unknown
4,4,5,creme soft soap - for hands & body,beauty & hygiene,bath & hand wash,nivea,162.0,162.0,bathing bars & soaps,4.4,Nivea Creme Soft Soap gives your skin the best...,Unknown


In [41]:
len(df)

27553

In [42]:
def split_quantity_pairs_safe(qty):
    if pd.isna(qty):
        return []
    
    parts = qty.split()
    result = []
    
    # Loop safely in steps of 2
    for i in range(0, len(parts) - 1, 2):  # len - 1 to avoid index out of range
        pair = f"{parts[i]} {parts[i+1]}"
        result.append(pair)
    
    return result


In [43]:

df['atomic_quantity'] = df['quantity'].apply(split_quantity_pairs_safe)
df_exploded = df.explode('atomic_quantity')


In [44]:
df.to_csv('Cleaned_BigBasket_Products.csv', index=False)